# step1 ：Download Data
已经手动下载

# step2 : Preparing Data
我们已经准备好了数据，所以第二步是对数据预处理，包括以下步骤
1. 加载特征数据：load_feat
2. 对数据进行增强，把特征移给前一“帧”和后一“帧”，得到多个平移版本（比如左移1帧、不移、右移1帧）
3. 把多个版本的数据拼接成为一个大数据

        **举例子**：
        
        3.1 假设你在看一个人“挥手”的动作，视频由 5 帧组成：
        第1帧：手放在身体旁边
        第2帧：手抬到半空中
        第3帧：手举到最高点（当前帧）
        第4帧：手开始放下
        第5帧：手回到身体旁边

        如果只用第3帧（手举到最高点）的数据，计算机可能只知道“手在某个位置”，但不知道这是一个“挥手”动作。
        这时候，shift 函数就像让计算机同时“偷看”前一帧（第2帧）和后一帧（第4帧）的特征，即把这三部分的数据通过复制，放在一个矩阵中，替换掉只有第三帧数据的矩阵，即结合前后的动作变化，计算机就能理解这是一个完整的“挥手”过程。

        对所有帧都这样处理，然后把他们拼接起来，你可以理解为，你看的视频都是帧拼接起来的，那么你制造的这个新的“视频”也要拼接起来。这就是concat_feat的作用

        3.2 那么对于数据的处理到底是怎么样的呢？
        定义张量x
        x = torch.tensor([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 9]])
        依据shift函数，当 n = -1 时，张量向上移动 1 个位置，用第 1 行进行填充，依据代码为
        left = x[0].repeat(1, 1)：n 是负数，-n 等于 1。因此，x[0].repeat(1, 1) 将第一行 [1, 2, 3] 重复 1 次，结果仍然是 [1, 2, 3]。

        right = x[:n]：因为 n 是负数，这表示索引从头到倒数第一个之前的所有行，所以 right = x[:-1] 结果是前两行 [[1, 2, 3], [4, 5, 6]]。
        把left和right拼接起来即变为
        tensor([[1, 2, 3],
                [1, 2, 3],
                [4, 5, 6]])
        如果你设置了n=-2,你可以理解为你觉得[1,2,3]这个特征很重要，所以你多复制了几次，创造了一个更加突出[1,2,3]这个特征的“帧”
        通过改变n，我们可以得到很多平移的版本，得到很多新的一“帧”，通过concat_feat把他们放在一起（比如直接拼在一起），随便举一个例子
        [
        第1帧：左移帧 + 当前帧 + 右移帧
        [1,2,3, 1,2,3, 1,2,3],  
        第2帧：左移帧 + 当前帧 + 右移帧
        [1,2,3, 1,2,3, 4,5,6],  
        第3帧：左移帧 + 当前帧 + 右移帧
        [1,2,3, 4,5,6, 4,5,6]   
        ]

4. preprocess_data：从各个文件加载进相应的数据

In [60]:
import os
import random
import pandas as pd
import torch
from tqdm import tqdm

def load_feat(path):
    feat = torch.load(path)
    return feat

def shift(x, n):
    if n < 0:
        left = x[0].repeat(-n, 1)
        right = x[:n]

    elif n > 0:
        right = x[-1].repeat(n, 1)
        left = x[n:]
    else:
        return x

    return torch.cat((left, right), dim=0)

def concat_feat(x, concat_n):
    assert concat_n % 2 == 1 # n must be odd
    if concat_n < 2:
        return x
    seq_len, feature_dim = x.size(0), x.size(1)
    x = x.repeat(1, concat_n)
    x = x.view(seq_len, concat_n, feature_dim).permute(1, 0, 2) # concat_n, seq_len, feature_dim
    mid = (concat_n // 2)
    for r_idx in range(1, mid+1):
        x[mid + r_idx, :] = shift(x[mid + r_idx], r_idx)
        x[mid - r_idx, :] = shift(x[mid - r_idx], -r_idx)

    return x.permute(1, 0, 2).view(seq_len, concat_n * feature_dim)

def preprocess_data(split, feat_dir, phone_path, concat_nframes, train_ratio=0.8, train_val_seed=1337):
    class_num = 41 # NOTE: pre-computed, should not need change
    mode = 'train' if (split == 'train' or split == 'val') else 'test'

    label_dict = {}
    if mode != 'test':
      phone_file = open(os.path.join(phone_path, f'{mode}_labels.txt')).readlines()

      for line in phone_file:
          line = line.strip('\n').split(' ')
          label_dict[line[0]] = [int(p) for p in line[1:]]

    if split == 'train' or split == 'val':
        # split training and validation data
        usage_list = open(os.path.join(phone_path, 'train_split.txt')).readlines()
        random.seed(train_val_seed)
        random.shuffle(usage_list)
        percent = int(len(usage_list) * train_ratio)
        usage_list = usage_list[:percent] if split == 'train' else usage_list[percent:]
    elif split == 'test':
        usage_list = open(os.path.join(phone_path, 'test_split.txt')).readlines()
    else:
        raise ValueError('Invalid \'split\' argument for dataset: PhoneDataset!')

    usage_list = [line.strip('\n') for line in usage_list]
    print('[Dataset] - # phone classes: ' + str(class_num) + ', number of utterances for ' + split + ': ' + str(len(usage_list)))

    max_len = 3000000
    X = torch.empty(max_len, 39 * concat_nframes)
    if mode != 'test':
      y = torch.empty(max_len, dtype=torch.long)

    idx = 0
    for i, fname in tqdm(enumerate(usage_list)):
        feat = load_feat(os.path.join(feat_dir, mode, f'{fname}.pt'))
        cur_len = len(feat)
        feat = concat_feat(feat, concat_nframes)
        if mode != 'test':
          label = torch.LongTensor(label_dict[fname])

        X[idx: idx + cur_len, :] = feat
        if mode != 'test':
          y[idx: idx + cur_len] = label

        idx += cur_len

    X = X[:idx, :]
    if mode != 'test':
      y = y[:idx]

    print(f'[INFO] {split} set')
    print(X.shape)
    if mode != 'test':
      print(y.shape)
      return X, y
    else:
      return X


# step3 ：定义数据集，即定义各种从文件读出数据后存储，利用索引查找的方法

In [61]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

class LibriDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = X
        if y is not None:
            self.label = torch.LongTensor(y)
        else:
            self.label = None
    
    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]
    
    def __len__(self):
        return len(self.data)

# step4 ：定义要使用的模型

In [62]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(BasicBlock, self).__init__()

        self.block = nn.Sequential(
            nn.Linear(input_dim, output_dim),
            nn.ReLU(),
        )
    
    def forward(self, x):
        x = self.block(x)
        return x

class Classifier(nn.Module):
    def __init__(self, input_dim, output_dim=41, hidden_layers=1, hidden_dim=256):
        super(Classifier, self).__init__()

        self.fc = nn.Sequential(
            BasicBlock(input_dim, hidden_dim),
            *[BasicBlock(hidden_dim, hidden_dim) for _ in range(hidden_layers)],
            nn.Linear(hidden_dim, output_dim)
        )
    
    def forward(self, x):
        x = self.fc(x)
        return x

# step5 ：定义超参数和seed

In [63]:
# data prarameters
concat_nframes = 1              # the number of frames to concat with, n must be odd (total 2k+1 = n frames)
train_ratio = 0.8               # the ratio of data used for training, the rest will be used for validation

# training parameters
seed = 0                        # random seed
batch_size = 512                # batch size
num_epoch = 5                   # the number of training epoch
learning_rate = 0.0001          # learning rate
model_path = './model.ckpt'     # the path where the checkpoint will be saved

# model parameters
input_dim = 39 * concat_nframes # the input dim of the model, you should not change the value
hidden_layers = 1               # the number of hidden layers
hidden_dim = 256                # the hidden dim

In [64]:
import numpy as np

#fix seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

# step6 ：准备数据集和模型

In [65]:
import gc

# fix random seed
same_seeds(seed)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'DEVICE: {device}')

# preprocess data
train_X, train_y = preprocess_data(split='train', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)
val_X, val_y = preprocess_data(split='val', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes, train_ratio=train_ratio)

# get dataset
train_set = LibriDataset(train_X, train_y)
val_set = LibriDataset(val_X, val_y)

# remove raw feature to save memory
del train_X, train_y, val_X, val_y
gc.collect()

# get dataloader
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False)

DEVICE: cuda
[Dataset] - # phone classes: 41, number of utterances for train: 2743


0it [00:00, ?it/s]/tmp/ipykernel_1653125/770864125.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  feat = torch.load(path)
2743it [00:01, 2413.94it/s]


[INFO] train set
torch.Size([1696055, 39])
torch.Size([1696055])
[Dataset] - # phone classes: 41, number of utterances for val: 686


686it [00:00, 2367.45it/s]


[INFO] val set
torch.Size([420739, 39])
torch.Size([420739])


In [66]:
# create model, define a loss function, and optimizer
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# step6 ：训练

In [67]:
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, batch in enumerate(tqdm(train_loader)):
        features, labels = batch
        features = features.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(features)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        train_acc += (train_pred.detach() == labels.detach()).sum().item()
        train_loss += loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, batch in enumerate(tqdm(val_loader)):
                features, labels = batch
                features = features.to(device)
                labels = labels.to(device)
                outputs = model(features)

                loss = criterion(outputs, labels)

                _, val_pred = torch.max(outputs, 1)
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


100%|██████████| 822/822 [00:02<00:00, 315.26it/s]


[001/005] Train Acc: 0.417492 Loss: 2.116242 | Val Acc: 0.435852 loss: 1.992612
saving model with acc 0.436


100%|██████████| 822/822 [00:04<00:00, 189.87it/s]


[002/005] Train Acc: 0.446777 Loss: 1.949217 | Val Acc: 0.444592 loss: 1.945544
saving model with acc 0.445


100%|██████████| 822/822 [00:02<00:00, 288.03it/s]


[003/005] Train Acc: 0.453577 Loss: 1.915422 | Val Acc: 0.449709 loss: 1.923863
saving model with acc 0.450


100%|██████████| 822/822 [00:04<00:00, 184.65it/s]


[004/005] Train Acc: 0.457353 Loss: 1.897430 | Val Acc: 0.451605 loss: 1.912621
saving model with acc 0.452


100%|██████████| 822/822 [00:02<00:00, 294.81it/s]

[005/005] Train Acc: 0.459758 Loss: 1.885291 | Val Acc: 0.454201 loss: 1.901589
saving model with acc 0.454


In [68]:
del train_loader, val_loader
gc.collect()

0

# step7 ：测试

In [69]:
# load data
test_X = preprocess_data(split='test', feat_dir='./libriphone/feat', phone_path='./libriphone', concat_nframes=concat_nframes)
test_set = LibriDataset(test_X, None)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# load model
model = Classifier(input_dim=input_dim, hidden_layers=hidden_layers, hidden_dim=hidden_dim).to(device)
model.load_state_dict(torch.load(model_path))

[Dataset] - # phone classes: 41, number of utterances for test: 857


0it [00:00, ?it/s]/tmp/ipykernel_1653125/770864125.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  feat = torch.load(path)
857it [00:00, 2820.97it/s]

[INFO] test set
torch.Size([527364, 39])


<All keys matched successfully>

In [70]:
# make prediction
test_acc=0.0
test_length=0
pred = np.array([], dtype=np.int32)

model.eval()
with torch.no_grad():
    for i, batch in enumerate(tqdm(test_loader)):
        features = batch
        features = features.to(device)

        outputs = model(features)

        _, test_pred = torch.max(outputs, 1)
        pred = np.concatenate((pred, test_pred.cpu().numpy()), axis=0)

# write prediction to a CSV file
with open('prediction.csv','w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(pred):
        f.write('{},{}\n'.format(i, y))

100%|██████████| 1031/1031 [00:02<00:00, 461.70it/s]
